# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of *C++* library.

First, import **AutoWIG**.

In [1]:
import autowig

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [2]:
%%bash --err error
conda build -q basic/conda/libbasic -c statiskit
conda install -y -q libbasic --use-local -c statiskit

scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/cpp/base.h" as "/home/pfernique/.miniconda/conda-bld/libbasic_1494317428769/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehol/include/basic/base.h"
Install file: "build/cpp/binomial.h" as "/home/pfernique/.miniconda/conda-bld/libbasic_1494317428769/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehol/include/basic/binomial.h"
Install file: "build/cpp/overload.h" as "/home/pfernique/.miniconda/conda-bld/libbasic_1494317428769/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl

Once the headers have been installed in the system, we parse them with relevant compilation flags.

In [3]:
%%time
import sys
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg, [sys.prefix + '/include/basic/overload.h',
                           sys.prefix + '/include/basic/binomial.h'],
                          ['-x', 'c++', '-std=c++11'],
                          silent = True)

CPU times: user 120 ms, sys: 40 ms, total: 160 ms
Wall time: 195 ms


Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is suitable.

In [4]:
%%time
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

CPU times: user 68 ms, sys: 0 ns, total: 68 ms
Wall time: 67.2 ms


In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [5]:
%%time
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = 'basic/src/py/_basic.cpp',
                             decorator = 'basic/src/py/basic/_basic.py',
                             prefix = 'wrapper_')

CPU times: user 40 ms, sys: 4 ms, total: 44 ms
Wall time: 45.3 ms


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [6]:
%%time
wrappers.write()

CPU times: user 356 ms, sys: 428 ms, total: 784 ms
Wall time: 1.94 s


Here is an example of the generated wrappers.
We here present the wrappers for the `BinomialDistribution` class.

In [7]:
%%bash
pygmentize basic/src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp

#include "_basic.h"



namespace autowig
{

}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::BinomialDistribution const volatile * get_pointer<class ::BinomialDistribution const volatile >(class ::BinomialDistribution const volatile *c) { return c; }
}
    #endif
#endif



void wrapper_4046a8421fe9587c9dfbc97778162c7d()
{

    double  (::BinomialDistribution::*method_pointer_3a3ff64f25e358a6a10b1cd3b3425b82)(unsigned int const) const = &::BinomialDistribution::pmf;
    double  (::BinomialDistribution::*method_pointer_3c97a500c9575c259d5cbdd76120ff4f)() const = &::BinomialDistribution::get_pi;
    void  (::BinomialDistribution::*method_pointer_d364a0529e33516f8ecbb7dcedd60aa0)(double const) = &::BinomialDistribution::set_pi;
    boost::python::class_< class ::BinomialDistribution, autowig::Held< class ::BinomialDistribution >::Type > class_4046a8421fe9587c9dfbc97778162c7d("BinomialDistribution", "", boost::python::no_init);
    class_4046a842

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [8]:
%%bash --err error
conda build -q basic/conda/python-basic -c statiskit
conda install -y -q python-basic --use-local -c statiskit --force

scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
g++ -o build/py/_basic.h.gch -x c++-header -c -fPIC -std=c++11 -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/pfernique/.miniconda/conda-bld/python-basic_1494317441975/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac/include -I/home/pfernique/.miniconda/conda-bld/python-basic_1494317441975/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac/include/python2.7 build/py/_basic.h
Creating 'build/py/response_file.rsp'
Install file: "build/cpp/base.h" as "/home/pfernique/.miniconda/conda-bld/python-basic_1494317441975/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [9]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [10]:
binomial.pmf(0)

0.5

In [11]:
binomial.pmf(1)

0.5

In [12]:
binomial.n = 0
binomial

In [13]:
binomial.pmf(0)

1.0

In [14]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print error.message
else:
    raise Exception('A `basic.ProbabilityError` should have been raise')

AttributeError: 'module' object has no attribute 'ProbabilityError'